<a href="https://colab.research.google.com/github/LucasLessa1/X-rayBodyPart/blob/davi1/Proj_CIS_Imagens_Raio_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Ajusting file


In [149]:
pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [150]:
import cv2
import pydicom
import pandas as pd
import os
import shutil
import time
import math
import gdown
import zipfile
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import os
from PIL import Image
from google.colab.patches import cv2_imshow
import string

In [151]:
def download(id): 
    url = 'https://drive.google.com/uc?id=' + str(id)
    gdown.download(url, output = None, quiet = False)

def unzip(path): #Função para unzip
    zip = zipfile.ZipFile(path)
    zip.extractall()
    zip.close()

In [152]:
# https://drive.google.com/file/d/1ev-r31j8oRzDlKM_toaeADO2psrA_XXm/view?usp=sharing

download('1ev-r31j8oRzDlKM_toaeADO2psrA_XXm')
unzip('/content/archive.zip')

Downloading...
From: https://drive.google.com/uc?id=1ev-r31j8oRzDlKM_toaeADO2psrA_XXm
To: /content/archive.zip
100%|██████████| 269M/269M [00:02<00:00, 123MB/s]


In [153]:
#os.chdir("/content/drive/MyDrive/RAIO-X/RAIO-X/archive.zip (Unzipped Files)")
path = os.getcwd() 
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv(os.path.join('/content/sample_submission.csv'))

In [154]:
#from google.colab import drive
#drive.mount('/content/drive')

In [155]:
test_df.head(5)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10001001190452685542...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,1.2.826.0.1.3680043.8.498.10022667601042710442...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2,1.2.826.0.1.3680043.8.498.10024395388921105474...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3,1.2.826.0.1.3680043.8.498.10026689165626095651...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4,1.2.826.0.1.3680043.8.498.10035936364561920980...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [156]:
A=train_df.head(1)['SOPInstanceUID']
print(A[0][50:])

00333618114258


#Pre-processing

In [157]:
print("Columns:")
print(train_df.columns, "\n")

print("Types in columns:")
print(train_df.dtypes, "\n")

print("Types in info:")
print(train_df.describe(), "\n")

Columns:
Index(['SOPInstanceUID', 'Target'], dtype='object') 

Types in columns:
SOPInstanceUID    object
Target            object
dtype: object 

Types in info:
                                           SOPInstanceUID Target
count                                                1738   1738
unique                                               1738     41
top     1.2.826.0.1.3680043.8.498.10025629581362719970...     3 
freq                                                    1    724 



In [158]:
#Let's  see our Target distribution

bodyparts = {
0 : 'Abdomen' ,
1 :'Ankle' ,
2 :'Cervical Spine',
3 : 'Chest' ,
4 :'Clavicles' ,
5 :'Elbow' ,
6 :'Feet' ,
7 : 'Finger' ,
8 : 'Forearm' ,
9 : 'Hand' ,
10 : 'Hip' ,
11 : 'Knee' ,
12 : 'Lower Leg' ,
13 : 'Lumbar Spine' ,
14 : 'Others' ,
15 :'Pelvis',
16 :'Shoulder' ,
17 :'Sinus' ,
18 : 'Skull' ,
19 : 'Thigh' ,
20 :'Thoracic Spine',
21: 'Wrist',
}

##Fixing folders

In [308]:
# example of horizontal shift image augmentation
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import random, time

def augment(files, exams, limit):
     for exam in exams: 
          # load the image
          img = load_img(f'/content/train_img/{files}/{exam}')
          # convert to numpy array
          data = img_to_array(img)
          # expand dimension to one sample
          samples = expand_dims(data, 0)
          # create image data augmentation generator
          datagen = ImageDataGenerator(horizontal_flip=True,rotation_range=90,brightness_range=[0.2, 1.5])
          # prepare iterator
          it = datagen.flow(samples, batch_size=1)
          for _ in range(limit):
            # generate batch of images
            batch = it.next()
            image = batch[0].astype('uint16')
            resized_img  = (np.maximum(image,0)/image.max())*255 
            im = Image.fromarray(resized_img.astype(np.uint8))
            im.save(f'/content/train_img/{files}/augmented_{exam}')



class Image_processing():
  def __init__(self):
    self.fileList = []

  def fixfolders(self, folder, df, dictionary):
    filelist = []
    for root, dirs, files in os.walk(f'/content/{str(folder)}'):
      for file in files:
        filelist.append(os.path.join(root,file))

    for filename in filelist:
      dicom = pydicom.dcmread(filename)
      img = dicom.pixel_array
      var = filename.split("/")
      var = var[-1]
      row = df.index[df['SOPInstanceUID']==f'{var[:-6]}'].tolist()
      target = df['Target'].iloc[row[0]]
      if len(target.strip()) > 2: 
        continue
      label = dictionary.get(int(target))
      resized_img  = (np.maximum(img,0)/img.max())*255 
      im = Image.fromarray(resized_img.astype(np.uint8))
      im.show()
      im.save(os.path.join(f'/content/{folder}_img', f'{label}', f'{var[:-6]}.png'))
 
  def createfolders(self, folder):
    os.mkdir(f'/content/{folder}/') 
    #os.chdir(f'/content/{folder}') 
    for bodypart in list(bodyparts.values()):
      if os.path.isdir(f'/content/{folder}/{bodypart}') == False:
        os.mkdir(f'/content/{folder}/{bodypart}')
      else:  
          pass 

  def process_imgs(self):
    for files in os.listdir('/content/train_img'):
      if len(os.listdir(f'/content/train_img/{files}')) < 100: 
        diff = len(os.listdir(f'/content/train_img/{files}')) 
        if diff >= 50 :
          exams = random.choices(os.listdir(f'/content/train_img/{files}'), k= diff)
          augment(files, exams, 1)
        else:
          if diff % 10 != 0: 
            #Quntos algarismos para próxima dezena
            ref1 = (math.ceil(diff / 10) * 10) - diff 
            for _ in range(ref1):
              exams = random.choices(os.listdir(f'/content/train_img/{files}'), k= ref1)
              augment(files, exams, ref1)
          else: 
            ref2 = 100 - (math.ceil(diff / 10) * 10)
            #pego 10 de amsmostra e multiplico a quantidade de vezes para somar 100 
            for _ in range(ref2):
              exams = random.choices(os.listdir(f'/content/train_img/{files}'), k= 10)
              ref2 = int(ref2/10)
              augment(files, exams, ref2)
       

In [310]:
#os.chdir("/content/drive/MyDrive/RAIO-X/RAIO-X/archive.zip (Unzipped Files)")
#path = os.getcwd()

#CRIAR PASTAS E SEPARAR AQUIVOS 
fix  =  Image_processing()
fix.createfolders('train_img')
fix.fixfolders('train', train_df, bodyparts)

In [311]:
fix.process_imgs()

In [314]:
for files in os.listdir('/content/train_img'):
  print(files)
  print(len(os.listdir(f'/content/train_img/{files}')))

Skull
17
Sinus
59
Lower Leg
20
Thigh
13
Hip
20
Lumbar Spine
40
Finger
30
Chest
724
Thoracic Spine
32
Cervical Spine
127
Knee
102
Others
120
Wrist
105
Abdomen
126
Ankle
93
Clavicles
10
Hand
79
Forearm
38
Elbow
52
Feet
110
Shoulder
48
Pelvis
109


In [309]:
#Excluir diretorios de Labels!!!!!!!!
#shutil.rmtree(f'/content/train_img/')

In [164]:
# DAQUI PRA BAIXO NÃO GARANTO NADA !!

In [165]:
def dcmtag2table(folder, list_of_tags):

    list_of_tags = list_of_tags.copy()
    items = []
    table = []
    filelist = []

    for root, dirs, files in os.walk(folder, topdown=False):
#Adiciona os caminhos de cada imagem na lista
        for name in files:
            filelist.append(os.path.join(root, name))
    aux= 0
    for _f in filelist:
      #Em cada caminho da imagem(.dcm)
        try:
            ds = pydicom.dcmread(_f, stop_before_pixels=True)
            #Pega o Dataset de cada imagem e ler no próximo for
            items = []
            items.append(_f)
            #Adiciono o caminho em "items"
            for _tag in list_of_tags:
                if _tag in ds:
                  
                    items.append(ds.data_element(_tag).value)
                    #Adiciono a features "PhotometricInterpretation", "BitsAllocated" and "SOPInstanceUID" em "items"
                else:
                    items.append("Not found")
            
            table.append(items)
            #Adiciona tudo em uma outra lista
        except:
            print("Skipping non-DICOM: " + _f)

    df = pd.DataFrame(table, columns=["Filename", "PhotometricInterpretation", "BitsAllocated","SOPInstanceUID"])
    #Crio um data set com todos os caminhos Photometric, Bits and SOP
    return df

*   SOP Instance UID - Unique Identification
*   SOP Class UID    - 

In [166]:
tags = ['PhotometricInterpretation','BitsAllocated', 'SOPInstanceUID' ]
dicom_tags_train =  dcmtag2table('/content/train', tags)
dicom_tags_test = dcmtag2table('/content/test', tags)

In [167]:
dicom_tags_train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID
0,/content/train/train/train/1121/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.75371875098288485781...
1,/content/train/train/train/350/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.88472775577462540945...
2,/content/train/train/train/87/1.2.826.0.1.3680...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.39286386395654026252...
3,/content/train/train/train/87/1.2.826.0.1.3680...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.56051445150509921910...
4,/content/train/train/train/378/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.50515155554468059374...
...,...,...,...,...
1733,/content/train/train/train/919/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12489154905016857920...
1734,/content/train/train/train/1046/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11043587047469807004...
1735,/content/train/train/train/1178/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.56914468087245411678...
1736,/content/train/train/train/1178/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.60288704388386519974...


In [168]:
print(dicom_tags_train.PhotometricInterpretation.value_counts())

MONOCHROME1    1659
MONOCHROME2      79
Name: PhotometricInterpretation, dtype: int64


In [169]:
train = dicom_tags_train.merge(train_df, on =  'SOPInstanceUID')
test = dicom_tags_test.merge(test_df,on =  'SOPInstanceUID')

In [170]:
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target
0,/content/train/train/train/1121/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.75371875098288485781...,3
1,/content/train/train/train/350/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.88472775577462540945...,0
2,/content/train/train/train/87/1.2.826.0.1.3680...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.39286386395654026252...,3
3,/content/train/train/train/87/1.2.826.0.1.3680...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.56051445150509921910...,3
4,/content/train/train/train/378/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.50515155554468059374...,3


In [171]:
target = list(train["Target"])

In [172]:
labels = ['Abdomen', 'Ankle', 'Cervical Spine',
       'Chest', 'Clavicles', 'Elbow', 'Feet', 'Finger', 'Forearm', 'Hand',
       'Hip', 'Knee', 'Lower Leg', 'Lumbar Spine', 'Others', 'Pelvis',
       'Shoulder', 'Sinus', 'Skull', 'Thigh', 'Thoracic Spine', 'Wrist']

In [173]:
#Create Label from target
def no_to_label(label):
    label_list_string = []
    trimed_label = label.rstrip()
    label_list = trimed_label.split(" ")
    label_list = [int(i) for i in label_list]
    for label in label_list:
        label_list_string.append(bodyparts[label])
    label_string = ' and '.join(label_list_string)
    return label_string

target_list = train['Target'].tolist()
label_column = []

for label in tqdm(target_list):
    label_string = no_to_label(label)
    label_column.append(label_string)

train['Label'] = label_column
train['Label'].unique()

100%|██████████| 1738/1738 [00:00<00:00, 563781.93it/s]


array(['Chest', 'Abdomen', 'Hip and Knee and Thigh', 'Thigh', 'Shoulder',
       'Cervical Spine', 'Lower Leg', 'Ankle and Feet', 'Pelvis',
       'Abdomen and Chest', 'Sinus', 'Others', 'Hip', 'Elbow',
       'Lumbar Spine and Thoracic Spine', 'Knee', 'Hand and Wrist',
       'Wrist', 'Feet', 'Lumbar Spine', 'Finger', 'Ankle', 'Clavicles',
       'Ankle and Knee and Lower Leg', 'Hand', 'Hip and Thigh', 'Skull',
       'Forearm and Shoulder', 'Knee and Lower Leg', 'Pelvis and Thigh',
       'Chest and Pelvis', 'Forearm', 'Ankle and Lower Leg',
       'Elbow and Forearm', 'Thoracic Spine', 'Chest and Feet',
       'Chest and Finger', 'Abdomen and Pelvis', 'Finger and Hand',
       'Cervical Spine and Lumbar Spine and Thoracic Spine',
       'Cervical Spine and Thoracic Spine'], dtype=object)

In [174]:
train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
0,/content/train/train/train/1121/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.75371875098288485781...,3,Chest
1,/content/train/train/train/350/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.88472775577462540945...,0,Abdomen
2,/content/train/train/train/87/1.2.826.0.1.3680...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.39286386395654026252...,3,Chest
3,/content/train/train/train/87/1.2.826.0.1.3680...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.56051445150509921910...,3,Chest
4,/content/train/train/train/378/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.50515155554468059374...,3,Chest
...,...,...,...,...,...,...
1733,/content/train/train/train/919/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12489154905016857920...,3,Chest
1734,/content/train/train/train/1046/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11043587047469807004...,3,Chest
1735,/content/train/train/train/1178/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.56914468087245411678...,3,Chest
1736,/content/train/train/train/1178/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.60288704388386519974...,3,Chest


In [175]:
train.Label.value_counts()

Chest                                                 724
Others                                                120
Knee                                                  102
Abdomen                                                80
Cervical Spine                                         77
Feet                                                   70
Pelvis                                                 67
Wrist                                                  63
Lumbar Spine and Thoracic Spine                        45
Hand and Wrist                                         42
Ankle                                                  41
Shoulder                                               40
Lumbar Spine                                           39
Hand                                                   31
Sinus                                                  23
Elbow                                                  23
Lower Leg                                              19
Hip           

In [176]:
train['Composto'] = [0 if x.count(' and ') == 0 else 1 for x in train.Label]
train.drop(train[train['Composto'] == 1].index, inplace=True)
train.drop(columns = ['Composto'], inplace=True)

In [177]:
for value in train.Label.unique():
  print(len(train[train.Label == value]))
  if len(train[train.Label == value]) < 100 : 
    for index, row in train.iterrows():
      dicom = pydicom.dcmread(train.Filename[index])
      img = dicom.pixel_array
      resized_img  = (np.maximum(img,0)/img.max())*255 
      im = Image.fromarray(resized_img.astype(np.uint8))
      im.show()

724
80
7
40
77
19
67
23
120
19
23
102
63
70
39
15
41
9
31
10
12
15
